In [64]:
from abc import ABCMeta,abstractmethod


class Clasificador:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
        # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error
        y = datos[:,-1]
        err = (y == pred)
        return 1-(sum(err)/len(y))


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):
        # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
        # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
        # y obtenemos el error en la particion de test i
        # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
        # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        pass

##############################################################################

class ClasificadorNaiveBayes(Clasificador):
    
    def __init__(self):
        self.prior_probs = []
        self.likelihoods = []
        self.means = []
        self.vars = []

    
    def _multinomialNB(self, xdata, ydata, feat_idx, diccionario):
        n_xi = len(diccionario[feat_idx])
        n_classes = len(diccionario[-1])
        theta_mtx = np.zeros((n_xi, n_classes))
        
        for value in diccionario[feat_idx]:
            feat_val_idx = diccionario[feat_idx][value]
            for class_name in diccionario[-1]:
                class_idx = diccionario[-1][class_name]
                theta_mtx[feat_val_idx][class_idx] = sum((xdata[:,feat_idx] == value)&(ydata == class_name)) / sum(ydata == class_name)
        
        return theta_mtx
                      

    
    def _gaussianNB(self, xdata, ydata, diccionario, atributosDiscretos):
        ndata, nfeat = xdata.shape
        # Define the Means and Vars Matrixes
        M = np.zeros((nfeat, len(diccionario[-1])))
        V = np.zeros((nfeat, len(diccionario[-1])))
        
        # We only compute those values for continuous features
        for j in range (nfeat):
            if (atributosDiscretos[j] == False):
                # We calculate the mean coditioned to each possible class
                # We calculate the variance conditioned to each possible class
                for k in range(len(diccionario[-1])):
                    m = []
                    for i in range(ndata):
                        if (ydata[i] == diccionario[-1][k]):                               
                            m.append(xdata[i][j])
                    M[j][k] = np.mean(m)
                    V[j][k] = np.var(m)
                    
        return M, V


    def entrenamiento(self,datostrain,atributosDiscretos,diccionario):
        xdata = datostrain[:,:-1] # all rows, all columns but last one
        ydata = datostrain[:,-1]  # all rows, just last column
        
        m, n = xdata.shape     # number of examples, number of features
        n_classes = len(diccionario[-1])  # number of different classes
        
        # Calculating prior probabilities
        self.prior_probs = np.zeros(n_classes) # initializing array of prior probs with zeros
        for class_name in diccionario[-1]:
            class_idx = diccionario[-1][class_name]
            self.prior_probs[class_idx] = sum((class_name == ydata))/m # P(y=i) = count(ydata==i)/len(ydata)
        
        likelihoods_list = []
        # Calculating likelihoods
        for feat_idx in range(n):
            if atributosDiscretos[feat_idx]:
                theta_mtx = self._multinomialNB(xdata, ydata, feat_idx, diccionario)

            likelihoods_list.append(theta_mtx)
        
        self.likelihoods = np.asarray(likelihoods_list)
        
        # Calculating Means and Vars matrixs for continuous data.
        self.means, self.vars = self._gaussianNB(xdata, ydata, diccionario, atributosDiscretos)

    

    # TODO: implementar
    def clasifica(self,datostest,atributosDiscretos,diccionario):
        xdata = datostest[:,:-1] # all rows, all columns but last one
        ydata = datostest[:,-1]  # all rows, just last column
        
        m, n = xdata.shape     # number of examples, number of features
        n_classes = len(diccionario[-1])  # number of different classes
        
        Pred = []
        Real = []
        ndata, nfeat = datostest.shape
        for i in range (ndata):
            auxpred = []
            for k in range(n_classes):
                aux = self.prior_probs[k]
                for j in range (n):
                    if(atributosDiscretos[j] == True):
                        aux = aux * self.likelihoods[j][diccionario[j].get(xdata[i][j])][k]
                    else:
                        aux = aux * norm.pdf((xdata[i][j] - self.means[j][k])/sqrt(self.vars[j][k]))/sqrt(self.vars[j][k])
                auxpred.append(aux)
            Pred.append(auxpred.index(max(auxpred)))
            Real.append(diccionario[-1].get(ydata[i]))
        
        return Pred, Real


In [65]:
dataset = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, nfeat = dataset.datos.shape
print(ndata)

958


In [75]:
NB = ClasificadorNaiveBayes()
NB.entrenamiento(dataset.datos[:int(ndata*(7/8))], dataset.nominalAtributos, dataset.diccionario)
pred, real = NB.clasifica(dataset.datos[int(ndata*(7/8)):], dataset.nominalAtributos, dataset.diccionario)
print(pred)
print("\n")
print(real)
print("\n")
cont = 0
for i in range(len(pred)):
    if(pred[i] == real[i]):
        cont += 1
print("Error: " + str(1 - cont/len(pred)))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Error: 0.8916666666666666


In [2]:
from sklearn.naive_bayes import MultinomialNB
from Datos import Datos
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [173]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
ttt_db.datos[:10]
X = ttt_db.datos[:,:-1]
y = ttt_db.datos[:,-1]

In [139]:
enc = OneHotEncoder(sparse=False)

In [140]:
X_2 = np.array(enc.fit_transform(X)) 

In [141]:
X.shape



(958, 9)

In [142]:
X_2.shape

(958, 27)

In [143]:
X_2

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [144]:
clf = MultinomialNB()

In [145]:
clf.fit(X_2,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [146]:
clf.score(X_2,y)

0.6983298538622129

In [211]:
my_clf = ClasificadorNaiveBayes()

In [208]:
ttt_db.diccionario

[{'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'b': 0, 'o': 1, 'x': 2},
 {'negative': 0, 'positive': 1}]

In [212]:
my_clf.entrenamiento(ttt_db.datos, ttt_db.nominalAtributos, ttt_db.diccionario)

In [213]:
my_clf.likelihoods[1]

array([[0.23493976, 0.27476038],
       [0.30421687, 0.3658147 ],
       [0.46084337, 0.35942492]])

In [214]:
my_clf.likelihoods[2]

array([[0.18975904, 0.22683706],
       [0.43975904, 0.30191693],
       [0.37048193, 0.47124601]])